In [1]:
from ovarian_utils import MetaPolygon
import numpy as np
import pandas as pd
from ovarian_utils import read_qupath_json
from shapely.strtree import STRtree
from pathlib import Path
import json
from shapely.geometry import Point, Polygon, mapping, shape, box
from shapely.ops import unary_union

import matplotlib.pyplot as plt
import random
plt.rcParams['figure.dpi'] = 500 # for high resolution figure


In [2]:
def write_qupath_noIDs_Polys(xmlsave, regions, region_labels, region_colors):
    trythis = '['
    if len(regions) == 0:
        pass
    else:
        for i in range(0, len(regions)):
            roi = regions[i]
            label = region_labels[i]
            trythis += json.dumps(
                {"type": "Feature", "id": "PathAnnotationObject", "geometry": mapping(roi),
                 "properties": {"classification": {"name": label + '_'+str(i), "colorRGB": region_colors[label]}, "isLocked": False,
                                "measurements": []}}, indent=4)
            if i < len(regions) - 1:
                trythis += ','

    trythis += ']'

    with open(xmlsave, 'w') as outfile:
        outfile.write(trythis)

In [6]:
bbox_json_dir = '/media/14TB/aarlova_ovarian/ovarian demo/demo_objDet_preds/'

segm_json_dir = '/media/14TB/aarlova_ovarian/ovarian demo/demo_segm_preds/'

save_dir = '/media/14TB/aarlova_ovarian/ovarian demo/demo_merged_preds/'


file_list = ['17133893']
print(len(file_list))
print(file_list)

1
['17133893']


In [9]:
df_master = pd.DataFrame()

failed = []

for i in range(len(file_list)):
    # try:
        
    slide_name = file_list[i]
    print('Working on', i, slide_name)
    print(bbox_json_dir + slide_name + '_fastRCNN.json')
    print(segm_json_dir + slide_name + '.json')

    # read in BBOX jsons
    bbox_regions, bbox_region_labels = read_qupath_json(bbox_json_dir + slide_name + '_fastRCNN.json')
    print('loaded bbox')

    # read in SEGMENTATION json
    segm_regions, segm_region_labels = read_qupath_json(segm_json_dir + slide_name + '.json')
    print('loaded segm')

    # convert BBOX and SEGMENTATION regions to MetaPolygons
    bbox_polygons = []
    for i in range(len(bbox_regions)):
        metapoly = MetaPolygon('Follicle', bbox_regions[i])
        bbox_polygons.append(metapoly)

    segm_polygons = []
    for k in range(len(segm_regions)):
        metapoly = MetaPolygon('Follicle', segm_regions[k])
        segm_polygons.append(metapoly)

    print('BBOX polygons', len(bbox_polygons), 'SEGM polygons', len(segm_polygons))


    ######################## compare SEGM and BBOX polygons:
    df_out = pd.DataFrame()

    tree = STRtree(bbox_polygons)

    matched_follicles = []
    unmatched_segm_follicles = []

    for poly in segm_polygons:
        query_geom = poly
        intersects = [o for o in tree.query(query_geom) if o.intersects(query_geom) and query_geom.intersection(o).area/o.area > 0.3]
        intersects = [o for o in intersects if o not in matched_follicles]# make sure each intersecting follicle only gets counted once
        if len(intersects) > 0:
            matched_follicles.extend(intersects)            
        else:
            if query_geom not in unmatched_segm_follicles: # make sure each unmatched segm follicle only gets counted once
                unmatched_segm_follicles.append(box(*query_geom.bounds))


    ####################### look for unmatched BBOX preds
    tree2 = STRtree(segm_polygons)
    unmatched_bbox_follicles = []

    for poly in bbox_polygons:
        query_geom = poly
        intersects = [o for o in tree2.query(query_geom) if o.intersects(query_geom)]
        if len(intersects) == 0 and query_geom not in unmatched_bbox_follicles: # make sure each unmatched bbox follicle only gets counted once
            unmatched_bbox_follicles.append(query_geom)

    df_out['Slide ID'] = [slide_name]
    df_out['Follicle match'] = [len(matched_follicles)]  # use this for binary, and follicle_match for multiclass seg (in Binary, all GT labels have been changed to Follicle)
    df_out['Total BBOX preds'] = [len(bbox_polygons)]
    df_out['Total SEGM preds'] = [len(segm_polygons)]
    df_out['SEGM preds not supported by BBOX preds'] = [len(unmatched_segm_follicles)]
    df_out['BBOX preds not supported by SEGM preds'] = [len(unmatched_bbox_follicles)]
    df_out['% BBOX preds coinciding with SEGM preds'] = [len(matched_follicles) / len(bbox_polygons)]

    df_master = pd.concat([df_master, df_out])

    region_colors = {'Follicle': -16274801}
    merged_follicles = matched_follicles + unmatched_segm_follicles
    labels = ['Follicle' for f in merged_follicles]
    print('Total Detections', len(merged_follicles))

    write_qupath_noIDs_Polys(save_dir + slide_name + '.json', merged_follicles, labels, region_colors)
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')

    # except:
    #     print('failed to merge')
    #     failed.append(slide_name)
    #     print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')

print(failed)


Working on 0 17133893
/media/14TB/aarlova_ovarian/ovarian demo/demo_objDet_preds/17133893_fastRCNN.json
/media/14TB/aarlova_ovarian/ovarian demo/demo_segm_preds/17133893.json
loaded bbox
loaded segm
BBOX polygons 481 SEGM polygons 391


/media/14TB/aarlova_ovarian/ovarian demo/ovarian_utils.py:23: ShapelyDeprecationWarning: Setting custom attributes on geometry objects is deprecated, and will raise an AttributeError in Shapely 2.0
  self.label = label
/media/14TB/aarlova_ovarian/ovarian demo/ovarian_utils.py:24: ShapelyDeprecationWarning: Setting custom attributes on geometry objects is deprecated, and will raise an AttributeError in Shapely 2.0
  self.id = id


Total Detections 398
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[]


In [10]:
df_master

,Slide ID,Follicle match,Total BBOX preds,Total SEGM preds,SEGM preds not supported by BBOX preds,BBOX preds not supported by SEGM preds,% BBOX preds coinciding with SEGM preds
0,17133893,55,481,391,343,404,0.114345
